In [203]:
import pathlib

import holoviews as hv
import hvplot.pandas # noqa
import numpy as np
import pandas as pd
import panel as pn
import datetime as dt
import calendar

pn.extension(sizing_mode='stretch_width')

In [204]:
data = {'name':['A', 'B', 'C', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'D'],
        'open_time': [dt.datetime(2021, 7, 1, 0, 0, 0), dt.datetime(2022, 7, 2, 0, 0), dt.datetime(2021, 7, 3, 0, 0),dt.datetime(2021, 7, 3, 0, 0), dt.datetime(2021, 8, 5, 0, 0, 0), dt.datetime(2021, 9, 11, 0, 0, 0), dt.datetime(2021, 10, 11, 0, 0, 0), dt.datetime(2022, 11, 11, 0, 0, 0), dt.datetime(2021, 5, 11, 0, 0, 0), dt.datetime(2022, 8, 11, 0, 0, 0), dt.datetime(2022, 5, 11, 0, 0, 0)],
        'count':[27, 24, 22, 32, 17, 34, 52, 62, 44, 88, 22],
        'size':[34, 56,67,78, 33, 55,69,79, 34,12, 23]
       }

df = pd.DataFrame(data)
df
df['open_date'] = pd.to_datetime(df['open_time']).dt.date
df


,name,open_time,count,size,open_date
0,A,2021-07-01,27,34,2021-07-01
1,B,2022-07-02,24,56,2022-07-02
2,C,2021-07-03,22,67,2021-07-03
3,D,2021-07-03,32,78,2021-07-03
4,A,2021-08-05,17,33,2021-08-05
5,B,2021-09-11,34,55,2021-09-11
6,C,2021-10-11,52,69,2021-10-11
7,D,2022-11-11,62,79,2022-11-11
8,A,2021-05-11,44,34,2021-05-11
9,B,2022-08-11,88,12,2022-08-11


In [205]:
df['year'] = pd.to_datetime(df['open_time']).dt.year
df['month'] = pd.to_datetime(df['open_time']).dt.month

df

,name,open_time,count,size,open_date,year,month
0,A,2021-07-01,27,34,2021-07-01,2021,7
1,B,2022-07-02,24,56,2022-07-02,2022,7
2,C,2021-07-03,22,67,2021-07-03,2021,7
3,D,2021-07-03,32,78,2021-07-03,2021,7
4,A,2021-08-05,17,33,2021-08-05,2021,8
5,B,2021-09-11,34,55,2021-09-11,2021,9
6,C,2021-10-11,52,69,2021-10-11,2021,10
7,D,2022-11-11,62,79,2022-11-11,2022,11
8,A,2021-05-11,44,34,2021-05-11,2021,5
9,B,2022-08-11,88,12,2022-08-11,2022,8


In [206]:
idf = df.interactive()



df.set_index('open_time', inplace=True)

In [207]:
years_slider = pn.widgets.IntSlider(name='Year', start=2021, end=2022, step=1)

months = pn.widgets.ToggleGroup(
    name='Months',
    options=[1, 2, 3, 4 ,5 ,6 ,7 ,8 ,9 ,10,11, 12], 
    value=[1, 2, 3, 4 ,5 ,6 ,7 ,8 ,9 ,10,11, 12],
    button_type='success')
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['count', 'size'],
    button_type='warning'
)


In [208]:
ipipeline = (
    idf[
        (idf.year == years_slider) & 
        (idf.month.isin(months))
    ]
    .groupby(['name', 'open_date'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='open_date')  
    .reset_index(drop=True)
)
ipipeline


In [209]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

'notebook'

In [210]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0","#58d8b0" ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
    pn.layout.HSpacer(height=50, background=PALETTE[3]),
)

Row(sizing_mode='stretch_width')
    [0] HSpacer(background='#ff6f69', height=50)
    [1] HSpacer(background='#ffcc5c', height=50)
    [2] HSpacer(background='#88d8b0', height=50)
    [3] HSpacer(background='#58d8b0', height=50)

In [211]:
if environment()=="server":
   theme="fast"
else:
   theme="simple"

In [212]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme=theme)
itable

In [213]:
ihvplot = ipipeline.hvplot(x='open_date', y=yaxis, by='name', color=PALETTE, line_width=6, height=400)
ihvplot

In [214]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards with hvplot .interactive', 
    sidebar=[years_slider, 'Manufacturers', months, 'Y axis' , yaxis],
    main=[ihvplot.panel(), itable.panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()
#template.servable();

Launching server at http://localhost:50831
